# CP Decomposition using ALS

In [102]:
import numpy as np
import tensorly as tl # Used for verification

## Khatri-Rao Product

In [103]:
"""
Input: A: Array of matrices whose Khatri-Rao product is to be calculated
Output: B: Khatri-Rao product of the matrices in A
"""
def khatri_rao_product(A):
    N = len(A)
    num_cols = A[0].shape[1]
    x_dim = 1
    for i in range(N):
        x_dim *= A[i].shape[0]
    B = np.empty((x_dim, num_cols))

    for i in range(num_cols):
        for j in range(N):
            if j == 0:
                kron_prod = A[j][:, i]
            else:
                kron_prod = np.kron(kron_prod, A[j][:, i])
        B[:, i] = kron_prod

    return B

## Test Khatri Rao Product

In [104]:
"""Khatri-Rao product using tensorly"""
print("Khatri-Rao product using tensorly")
A = [np.random.rand(2, 3), np.random.rand(1, 3), np.random.rand(3, 3)]
Result = tl.tenalg.khatri_rao(A)
print("Result Shape: ", Result.shape)
print("Result:")
print(Result)

"""Khatri Rao product using khatri_rao_product()"""
print("\nKhatri Rao product using khatri_rao_product()")
B = khatri_rao_product(A)
print("Result Shape: ", B.shape)
print("Result:")
print(B)

Khatri-Rao product using tensorly
Result Shape:  (6, 3)
Result:
[[0.02377144 0.06998371 0.01929836]
 [0.0069152  0.04740797 0.08584184]
 [0.00056316 0.01947196 0.04073857]
 [0.03495267 0.36171755 0.02027669]
 [0.01016786 0.24503266 0.09019357]
 [0.00082805 0.10064269 0.04280381]]

Khatri Rao product using khatri_rao_product()
Result Shape:  (6, 3)
Result:
[[0.02377144 0.06998371 0.01929836]
 [0.0069152  0.04740797 0.08584184]
 [0.00056316 0.01947196 0.04073857]
 [0.03495267 0.36171755 0.02027669]
 [0.01016786 0.24503266 0.09019357]
 [0.00082805 0.10064269 0.04280381]]


## Mode-N Matricization

In [105]:
"""
Input:  1) tensor: Input tensor
        2) n: mode along which to matricize the tensor (mode is 0-indexed)
Output: matrix: n-mode matricization of the tensor
"""
# Take mode = 0 for the first mode, mode = 1 for the second mode, ...., mode = n-1 for nth mode
def mode_n_matricization(tensor, n):
    mode = n
    # Get the size of the original tensor
    sz = tensor.shape
    # print(f"tensor size: {sz}")

    # Permute the dimensions of the tensor to bring the chosen mode to the front
    # This will make it easy to reshape the tensor into a matrix along the chosen mode
    permuted_dimensions = list(range(len(sz)))
#     print('Before Permuation, dimensions: ', permuted_dimensions)
    permuted_dimensions.remove(mode)
    permuted_dimensions.insert(0, mode)
#     print('After Permuation, dimensions: ', permuted_dimensions)
    permuted_tensor = tensor.transpose(*permuted_dimensions)
#     print(f"permuted tensor size: {permuted_tensor.size()}")

    # Reshape the permuted tensor into a matrix along the chosen mode
    matrix = permuted_tensor.reshape(sz[mode], -1)
#     print(f"matrix size: {matrix.size()}")

#     print(f"n-mode matricization along mode {mode}:")
#     print(matrix)

    return matrix

## Test Mode-N Matricization

In [106]:
my_tensor = np.empty((2,3,4))
my_tensor[0, :, :] = np.array([[1,4,7,10], [2,5,8,11], [3,6,9,12]])
my_tensor[1, :, :] = np.array([[13,16,19,22], [14,17,20,23], [15,18,21,24]])
print(f"Original Tensor: {my_tensor.shape}")

for n in range(len(my_tensor.shape)):
    print(f"\nn-mode matricization along mode {n}:")
    matrix = mode_n_matricization(my_tensor, n)
    print(matrix)

Original Tensor: (2, 3, 4)

n-mode matricization along mode 0:
[[ 1.  4.  7. 10.  2.  5.  8. 11.  3.  6.  9. 12.]
 [13. 16. 19. 22. 14. 17. 20. 23. 15. 18. 21. 24.]]

n-mode matricization along mode 1:
[[ 1.  4.  7. 10. 13. 16. 19. 22.]
 [ 2.  5.  8. 11. 14. 17. 20. 23.]
 [ 3.  6.  9. 12. 15. 18. 21. 24.]]

n-mode matricization along mode 2:
[[ 1.  2.  3. 13. 14. 15.]
 [ 4.  5.  6. 16. 17. 18.]
 [ 7.  8.  9. 19. 20. 21.]
 [10. 11. 12. 22. 23. 24.]]


## Einsum functions

In [107]:
"""
Input: N: Length of Array of factor matrices
Output: equation: equation string for torch.einsum()
"""
def einsum_equation(N):
    equation = ''    
    for i in range(N):
        if(i != 0):
            equation += ','
        equation += f'{chr(97 + i)}z'

    equation += '->'

    for i in range(N):
        equation += f'{chr(97 + i)}'
    # print(f"equation: {equation}")
    return equation

A = [np.random.rand(2, 3), np.random.rand(1, 3), np.random.rand(3, 3)]
eqn = einsum_equation(3)
print(f"equation: {eqn}")

equation: az,bz,cz->abc


## CP-ALS

In [108]:
"""
Input:  1) T: Tensor of size (s_1, s_2, ..., s_N)
        2) R: CP-Rank of the Tensor
Output: 1) A: Array of N factor matrices of size (s_i, R)
"""
def CP_ALS(T, R, max_iter=100, tol=1e-5):
    N = len(T.shape)
    sz = T.shape
    A = []
    for n in range(N):
        An = np.random.rand(sz[n], R)
        A.append(An)

    # Iterate until convergence
    for iter in range(max_iter):
        prev_factors = A.copy()

        for n in range(N):
            # Calculate the Khatri-Rao product of all the factor matrices except the n-th factor matrix
            B = khatri_rao_product(A[:n] + A[n+1:])
            B_pinv = np.linalg.pinv(B)

            # Calculate the n-mode matricization of the tensor
            Tn = mode_n_matricization(T, n)
            
            # Calculate the n-th factor matrix
            AnT = np.matmul(B_pinv, Tn.T)
            A[n] = AnT.T

        # Check for convergence
        if all(np.linalg.norm(A[i] - prev_factors[i]) < tol for i in range(T.ndim)):
            print(f"Converged at iteration {iter}")
            break

    return A  
    

## Test CP-ALS

In [109]:
"""CP Decomposition using tensorly"""
print("CP Decomposition using tensorly")
tensor = np.random.rand(3, 4, 5, 6)
rank = 3

print(f"Rank: {rank}")
print(f"Tensor: {tensor.shape}")
print('Original Tensor:')
print(tensor)

cp_decomp = tl.decomposition.CP(rank)
weights, factor_matrices = cp_decomp.fit_transform(tensor)

print('\nFactor Matrices:')
for i in range(len(factor_matrices)):
    print("\nFactor Matrix {} Shape: {}".format(i+1, factor_matrices[i].shape))
    print("Factor Matrix {}: ".format(i+1))
    print(factor_matrices[i])

reconstructed_tensor = tl.cp_tensor.cp_to_tensor((weights, factor_matrices))

# Norm difference between original tensor and reconstructed tensor
print('\nNorm difference between original tensor and reconstructed tensor: ', np.linalg.norm(tensor - reconstructed_tensor).item())


"""CP Decomposition using CP_ALS()"""
print("\nCP Decomposition using CP_ALS()")
A = CP_ALS(tensor, rank)

print('Factor Matrices:')
for i in range(len(A)):
    print("\nFactor Matrix {} Shape: {}".format(i+1, A[i].shape))
    print("Factor Matrix {}: ".format(i+1))
    print(A[i])

# Norm difference between original tensor and reconstructed tensor
eqn = einsum_equation(len(A))
print('\nNorm difference between original tensor and reconstructed tensor: ', np.linalg.norm(tensor - np.einsum(eqn, *A)).item())

CP Decomposition using tensorly
Rank: 3
Tensor: (3, 4, 5, 6)
Original Tensor:
[[[[0.3150327  0.68113947 0.12319929 0.40778778 0.19947787 0.98984584]
   [0.62717503 0.51992035 0.1644339  0.00344873 0.85917062 0.3877752 ]
   [0.63332659 0.33395555 0.61819967 0.44270185 0.71366019 0.65564638]
   [0.45548691 0.61447653 0.35846854 0.67478495 0.74672007 0.70473319]
   [0.7368089  0.58120423 0.09395995 0.52039795 0.31616529 0.87037313]]

  [[0.5390519  0.20814357 0.75704737 0.67123209 0.70951103 0.67613149]
   [0.4801499  0.10454754 0.72256855 0.21880833 0.69563023 0.59380856]
   [0.31137263 0.60822319 0.74364321 0.46886271 0.20485218 0.53413002]
   [0.78471342 0.29397967 0.04111    0.33106264 0.50762267 0.29541189]
   [0.32310153 0.89735278 0.31619723 0.62430953 0.59218469 0.01374551]]

  [[0.1269677  0.7126624  0.01826591 0.71275865 0.67361888 0.52091891]
   [0.39773889 0.87387116 0.98097626 0.47561128 0.28686742 0.32149458]
   [0.15236001 0.00148995 0.45085868 0.13110797 0.43253446 0.25018